In [1]:
from datetime import date, timedelta
import datetime as dt
from dateutil.parser import parse
from typing import Sequence
from pathlib import Path
import sh 
import gc

from PIL import Image
# import cv2


import pandas as pd
import numpy as np
import xarray as xr

import matplotlib.pyplot as plt
from matplotlib import figure
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
import matplotlib.cm         as cm
import matplotlib.colors     as mcolors
import matplotlib.patches    as mpatches
import matplotlib.transforms as mtransforms
import matplotlib.dates      as mdates
import matplotlib

from netCDF4 import Dataset
import geomagdata as gi

import pylab
from pylab import *
import pywt 

import scipy.io        as scio
import scipy.constants as const
from   scipy.fftpack   import fft
from   scipy.stats     import chi2



In [2]:
def FFT(Fs, data):
    L = len(data)

    N = np.power(2, np.ceil(np.log2(L)))
    N = int(N)

    FFT_y = np.abs(fft(data,N))/L*2
    Fre   = np.arange(int(N/2))*Fs/N
    FFT_y = FFT_y[range(int(N/2))]
    
    return Fre, FFT_y

# 计算lags阶以内的自相关系数，返回lags个值，将序列均值、标准差视为不变
def autocorrelation(x,lags):
    n = len(x)
    x = np.array(x)
    variance = x.var()
    x = x-x.mean()
    result = np.correlate(x, x, mode = 'full')[-n+1:-n+lags+1]/\
    (variance*(np.arange(n-1,n-1-lags,-1)))
    return result




In [3]:
def format_func(x, 
                pos=None
               ):
    x = mdates.num2date(x)
    
    if x.day == 15 and x.hour == 13:
        fmt = '%H\n%y/%m/%d'
    elif x.hour == 13:
        fmt = '%H\n%m/%d'
    else:
        fmt = '%H'
    label = x.strftime(fmt)
    
    
    return label


def add_right_cax(ax, pad, width):
    '''
    在一个ax右边追加与之等高的cax.
    pad是cax与ax的间距,width是cax的宽度.
    '''
    axpos = ax.get_position()
    caxpos = mtransforms.Bbox.from_extents(
        axpos.x1 + pad,
        axpos.y0,
        axpos.x1 + pad + width,
        axpos.y1
    )
    cax = ax.figure.add_axes(caxpos)

    return cax

In [4]:
def getAEindex_daily(date = dt.datetime(2022, 8, 31),
                     AEdir = Path(f'/run/media/echoo/TOSHIBA EXT/space_weather/AEindex/')
                    ):
    yyyymm = date.strftime('%Y%m')
    yyyymmdd = date.strftime('%Y%m%d')

    # https://wdc.kugi.kyoto-u.ac.jp/ae_realtime/index.html
    webpage = Path(f'https://wdc.kugi.kyoto-u.ac.jp/ae_realtime')
    AEonline = webpage/f'{yyyymm}'/f'rtae_{yyyymmdd}.png'
    AEfile = AEdir/f'rtae_{yyyymmdd}.png'
    
    if Path.is_file(AEfile):
        pass
    else:
        sh.touch(AEfile)
        sh.curl('-o', AEfile, AEonline)

    AEimage = Image.open(AEfile)
#     AEimage
    
    pixels = np.array(AEimage)[::-1]
    vspan = range(82, 196)
    hspan = range(80, 650)
    rpixel = 0
    wpixel = 255
    graypixel = 187
    
    scale = 2000/len(vspan)       # units: nT
    
    AEpixels = pixels[vspan][:,hspan][:,:,rpixel]

#     AEpixels = np.where(AEpixels == wpixel   , np.NaN, AEpixels)
#     AEpixels = np.where(AEpixels == graypixel, np.NaN, AEpixels)
#     plt.contour(AEpixels)
#     AEpixels.shape
    
    AEpixels = np.where(AEpixels == wpixel, 
                        None, 
                        AEpixels
                       )
    AEpixels = np.where(AEpixels == graypixel, 
                        None, 
                        AEpixels
                       )
    
    tlength = AEpixels.shape[1]
    AEedge = np.zeros(tlength)
#     AEedge.shape
    for  t, plh in enumerate(AEpixels[0][:]):
        for v, plv in enumerate(AEpixels[::-1, t]):
            if plv is not None :
                AEedge[t] = (len(vspan)-v) * scale
    #             print(h, AE[h])
                break
            AEedge[t] = np.NaN
            continue
    
    AEedge = np.where(AEedge > 1500, np.NaN, AEedge)
#     plt.plot(AEedge)
#     plt.ylim(0,2000)


    start = date
    end   = start + timedelta(days = 1)
    tspan = pd.date_range(start = start, 
                          periods = 1440, 
                          freq = 'T'         # frequency: 1 min
                         )    

    tseq = np.arange( len(tspan) ) * len(hspan) / len(tspan)    # for interp
    AE_interp = np.interp(tseq, 
                          np.arange(tlength), 
                          AEedge, 
                          left = True, 
                          right = True
                         )

    data = {'time':tspan,
            'AE'  :AE_interp
           }
    AEdframe = pd.DataFrame(data)
    AEdframe['AE'] = AEdframe['AE'].interpolate()
    return AEdframe

# AE = getAEindex_daily(datetime(2022,8,27), AEdir)

In [5]:
def plotF107(ax, indices, mark = 'a'):
#     ticks = [0, 12]
#     ax.xaxis.set_major_locator(mdates.HourLocator(ticks))
#     # ax.xaxis.set_minor_locator(mdates.Locator())
#     ax.xaxis.set_major_formatter(format_func)
    ax.set_xticks([])

    colors     = ['xkcd:pastel pink', 'xkcd:pinkish red']
    thresholds = [120]
    control    = [0] + thresholds + [1000]
    thresholds_spans = [i for i in zip(control, control[1:])]
    dates = indices.index
    
#     ax.set_xlim(dates[0], dates[-1])
    ax.set_ylim(90, 240)
    ax.set_yticks(np.arange(90, 270, 30))

    for color , thresholds_span in zip(colors, thresholds_spans):
        low, high = thresholds_span
        line_data = np.ma.masked_array(indices['f107s'].values, 
                                       mask = (indices['f107s'].values<low)|(indices['f107s'].values>high)
                                      )
        ax.plot(dates, 
                line_data, 
                color, 
                marker='.', 
                markersize = 3.5,
                linewidth = 0
               )

    for threshold in thresholds:
        ax.axhline(threshold,
                   linestyle = '--',
                   color = 'grey',
                   alpha = 0.3,
                   zorder = -100
                  )
        
    ax.set_ylabel("F10.7 (sfu)")
    ax.set_title(f'{mark})', loc = 'left')
#     ax.set_xlabel("time (LT)")
    # ax.grid(True)

    # fig.savefig('2015.png', bbox_inches='tight')

#     plt.show()

In [6]:
# help(plt.title)

In [7]:
def plotAE(ax, AEindex, mark = 'b'):
#     ticks = [0,12]
#     ax.xaxis.set_major_locator(mdates.HourLocator(ticks))
#     # ax.xaxis.set_minor_locator(mdates.Locator())
#     ax.xaxis.set_major_formatter(format_func)
    ax.set_xticks([])

    colors = ['mistyrose', 'crimson']
    thresholds = [500]
    control    = [0] + thresholds + [2000]
    thresholds_spans = [i for i in zip(control, control[1:])]
    
    ax.set_ylim(0,1500)
    ax.set_yticks(np.arange(0,2000,500))

    for color , thresholds_span in zip(colors, thresholds_spans):
        low, high = thresholds_span
        AEplt     = AEindex['AE'].values
        tplt      = AEindex['time']
        line_data = np.ma.masked_array(AEplt, 
                                       mask = (AEplt<low)|(AEplt>high)
                                      )
        ax.plot(tplt, 
                 line_data, 
                 color, 
                 marker = ',',
    #              linewidth = 1
                )

    for threshold in thresholds:
        ax.axhline(threshold,
                    linestyle = '--',
                    color = 'grey',
                    alpha = 0.3,
                    zorder = -100
                  )
    ax.set_ylabel("AE index (nT)")
    ax.set_title(f'{mark})', loc = 'left')
#     ax.set_xlabel("time (LT)")
    # ax.grid(True)

    # fig.savefig('2015.png', bbox_inches='tight')

#     plt.show()


In [8]:
def plotKp(ax, indices, mark = 'c'):
    dates = indices.index
    
#     ticks = [8, 20]
#     ax.xaxis.set_major_locator(mdates.HourLocator(ticks))
#     # ax.xaxis.set_minor_locator(mdates.Locator())
#     ax.xaxis.set_major_formatter(format_func)
    # inds['Ap'].plot(ax=ax)
    
    ax.set_ylim(0, 9)
    ax.set_yticks(np.arange(1,10,2))
    ax.set_xticks([])
    ax.bar(dates, 
            indices['Kp'].values, 
            width = 0.1, 
            color = np.where(indices['Kp']>3, 
                             'red', 
                             'lavenderblush'
                            )
    #         color = np.where(indices['Kp']>3, 'royalblue', 'aliceblue'),
    #         edgecolor = 'aliceblue',
    #         color = 'dodgerblue'
           )
    # indices['Kp'].plot(ax = ax, marker = '.')  # , marker='.'
    ax.axhline(3,
               linestyle = '--',
               color = 'grey',
               alpha = 0.3,
               zorder = -100
             )
    ax.set_ylabel("Kp index")
    ax.set_title(f'{mark})', loc = 'left')
#     ax.set_xlabel("time (LT)")
    # ax.grid(True)

    # fig.savefig('2015.png', bbox_inches='tight')

#     plt.show()

In [9]:
# period = 86400    # test
def plot_TECs_at_station(ax, 
                         sTEC,
                         zeniths,
                         time_system = 'UTC',
                         mark = 'd'
                        ):
    
    colors = ['orangered',
#               'crimson',
              'gold', 
              'aquamarine', 
#               'mediumaquamarine',
              'dodgerblue',
              'greenyellow',
#               'darkviolet'
             ]
    
    eleDegrees = 90 - np.degrees(zeniths)
    
    for i, sv in enumerate(sTEC.sv):
        ax.plot(sTEC.time.values[::10], 
                sTEC.sel(sv = sv)[::10],
                color = colors[i],
#                 marker = ',',
                linestyle = "-", 
                linewidth = '0.8',
                label = f'{sv.values} ' 
                        f'{eleDegrees.sel(sv = sv).values:5.2f}' 
                        f'\N{DEGREE SIGN}'
               )
    
#     legend(labelcolor = 'linecolor')
    legend()
    
#     ticks = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24]
#     ax.xaxis.set_major_locator(mdates.HourLocator(ticks))
#     ax.xaxis.set_minor_locator(mdates.HourLocator())
#     ax.xaxis.set_major_formatter(format_func)
    
    ax.set_ylim(0,200)
    ax.set_xlabel(f'{time_system} time')
    ax.set_ylabel(f'TEC / {sTEC.units}')
    
#     plt.title(f'{sTEC.observer.values} {sTEC.name}')
#     plt.savefig(f'{sTEC.observer.values}_{sTEC.name}.png',
#                 bbox_inches='tight', 
#                )

# plot_TECs_at_station(nc.sTEC_smth.sel(observer = 'GSLZ'),
#                      nc.zenith.sel(observer = 'GSLZ')
#                     )             # test

In [10]:
def plotTECnorm(ax, time, data, mark = 'd'):
    ax.plot(time, 
            data, 
            color = 'navy',
            marker = '.', 
            markersize = 0.05,
            linewidth = 0
           )
#     ax.set_xlabel('time (LT)')
    ax.set_ylabel('vTEC (TECU)')
    ax.set_xticks([])
    
    ax.set_ylim(-50, 50)
    ax.set_yticks(np.arange(-40, 60, 20))
    ax.set_title(f'{mark}) detrended vTEC ', loc = 'left')

#     ax.set_title(f'a) {data.observer.values} {data.observer.sv.values}')
#     ticks = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24]
#     f_ax1.xaxis.set_major_locator(mdates.HourLocator(ticks))
#     f_ax1.xaxis.set_minor_locator(mdates.HourLocator())
#     f_ax1.xaxis.set_major_formatter(format_func)
#     f_ax1.set_xlim(time[0], time[-1])

In [11]:
def plotCWT(ax, time, periods, frequencies, cwtmatr, 
            mark = 'e', 
            levels = np.arange(0.01, 0.25, 0.02)
           ):
    

#     levels = np.arange(1, 25, 1.5)    # low frequencies

#     levels = np.arange(0.2, 2.7, 0.1)  # medium frequencies
#     levels = np.arange(0.3, 2.2, 0.1)  # high frequencies
#     levels = np.arange(0.1, 1.5, 0.1)  # higher frequencies
#     levels = np.arange(0.01, 0.25, 0.02)  # muchhigher frequencies


#     cmap = plt.get_cmap('GnBu_r')
    cmap = plt.get_cmap('seismic')
#     cmap = plt.get_cmap('bwr')
    clipcolors = cmap(np.linspace(0, 1, 256))
    new_cmap = ListedColormap(clipcolors[128:20:-1])
    cax = add_right_cax(ax, pad = 0.04, width = 0.02)
    im = ax.contourf(time, 
#                      periods/60/60, 
                     frequencies,
                     np.abs(cwtmatr), 
                     levels, 
                     cmap = new_cmap, 
                     alpha = 0.75, 
                     extend = 'both'
                    )
#     ax.contour(time, 
# #                periods/60/60, 
#                frequencies,
#                np.abs(cwtmatr), 
#                levels[::2],
#                cmap = new_cmap,
#                alpha = 0.1,
#                linewidths = 0.5
#                )

    cbar = fig.colorbar(im, cax=cax)

#     ax.contourf(time, 
#                    periods/60/60,
#                    coimatr, 
#                    alpha = 0.7, 
#                    cmap = cmap)
#     ax.invert_yaxis()
#     ax.set_yscale('log', base =2)
#     ax.set_xlabel(u'time (LT)')
#     ax.set_ylabel('periods (hour)')
    ax.set_ylabel('frequencies (Hz)')
    ax.set_xticks([])
    ax.set_title(f'{mark}) CWT', loc = 'left')

#     ax.xaxis.set_major_locator(mdates.HourLocator(ticks))
#     ax.xaxis.set_minor_locator(mdates.HourLocator())
#     ax.xaxis.set_major_formatter(format_func)


# plt.savefig(f"waveletTransform_"
#             f"{obs['date']}_"
#             f"{data.observer.values}_{data.observer.sv.values}.png", 
#             bbox_inches='tight', 
#             dpi = 500
#            )
# plt.show()

In [12]:
# start = datetime(2022, 8, 22)
# end   = datetime(2022, 8, 29)
# timezone = 8
start = dt.datetime(2022, 1, 15)
end   = dt.datetime(2022, 1, 17)
timezone = 13
dates = pd.date_range(start, end, freq="3H")

try:
    indices = gi.get_indices(dates, smoothdays = 3)
except ConnectionError as e:
    pytest.skip(f"possible timeout error {e}")

pasttime = dt.datetime(2017, 1, 1)
if indices["resolution"][0] == "d":
    pasttime += timedelta(hours=1, minutes=30)

indices.index += timedelta(hours = timezone)

In [13]:
AEdir = Path(f'/run/media/echoo/TOSHIBA EXT/space_weather/AEindex/')

is_first_day = True
for date in pd.date_range(start, 
                          end-timedelta(days=1),
                          freq = '1D'
                         ):
    AE = getAEindex_daily(date, AEdir)
#     plt.ylim(0,2000)
#     plt.plot(AE['AE'])
    if is_first_day:
        AEindex = AE
    else:
        AEindex = pd.concat([AEindex, AE], 
                            axis = 0 
                           )
    is_first_day = False

AEindex['time'] += timedelta(hours = timezone) 

In [17]:
obs = {}

event_date       = datetime.date(2022,1,15).strftime('%Y_%m_%d')
epicenter        = 'Tonga'
src_dir  = Path(f'/run/media/echoo/TOSHIBA EXT/GNSS/{event_date}_{epicenter}')

save_dir  = Path(f'/home/echoo/echoo/mycode/PLOT/{event_date}_{epicenter}')

# save_dir
if not Path.is_dir(save_dir):
    sh.mkdir(save_dir)
# 
#     C01 - 地球静止轨道140.0°E，高度35807×35782公里，倾角1.6°
#     C02 - 地球静止轨道84.07°E，高度35803×35783公里，倾角1.7°
#     C03 - 地球静止轨道110.45°E，高度35854.3×35885.9公里，倾角1.7°
#     C04 - 地球静止轨道159.98°E，高度35815×35772公里，倾角0.6°
#     C05 - 地球静止轨道58.71°E，高度35801×35786公里，倾角1.4°

svs = [ 'C01', 'C03', 'C02', 'C05']


fc              = 1
fband           = 6
wavelet_medium         = f'cmor{fband}-{fc}'   # medium - higher

fc              = 10
fband           = 6
wavelet_higher  = f'cmor{fband}-{fc}'    # muchhigher

wavelet_low     = 'cgau8'    # low

wavelet         = wavelet_higher

drelation       = 2.32

sampling_period = 1.0 / 15 / 60            # units: 15min
sampling_rate   = 1.0 / sampling_period    # units: 1/15min
# totalfreqc      = 256
totalfreqc      = 128

# fc              = pywt.central_frequency(wavelet)
# cparam          = 2 * fc * totalscale
# scales          = cparam / np.arange(totalscale, 0, -1)

# max_power       = -11          # low frequencies
# min_power       = -13

# max_power       = -10         # medium frequencies
# min_power       = -11

# max_power       = -9         # high frequencies
# min_power       = -10

# max_power       = -8         # higher frequencies
# min_power       = -9

max_power       = -7         # muchhigher frequencies
min_power       = -8

interval        = (max_power - min_power) / totalfreqc 
fourier_freqces = 2 ** np.arange(min_power, max_power, interval) 
scales          = pywt.frequency2scale(wavelet, fourier_freqces)

observers = set()
observer_svs = {}

is_first_day = True
for date in pd.date_range(start+timedelta(days=1), 
                          end-timedelta(days=1), 
                          freq="D"
                         ):
    obs['year']      = date.strftime('%y')
    obs['doy']       = date.strftime('%j')
    obs['samp_rate'] = '01s'

    # src_dir  = Path(f"/run/media/echoo/TOSHIBA EXT")

    TEC_dir  = src_dir/obs['doy']/'TEC'
    TEC_file = f"TEC_{obs['doy']}_{obs['samp_rate']}_{obs['year']}.nc"
    TEC_path = TEC_dir/TEC_file
    
    nc = xr.open_dataset(TEC_path)
    if is_first_day:
        observers = set(nc.observer.values)
    else:
        observers = observers.intersection(set(nc.observer.values) )
        
    for observer in observers:
        if is_first_day:
            observer_svs[observer] = svs.copy()
        for sv in svs:
            sTEC   = nc.sTEC_smth.sel(observer = observer, sv = sv).dropna(dim = 'time') 
            zenith = nc.zenith.sel(observer = observer, sv = sv)
            
            if len(sTEC) < 40000:
                observer_svs[observer].remove(sv)
            try:
                elevation  = 90 - np.degrees(zenith.values)
                if elevation < 32:
                    observer_svs[observer].remove(sv)
            except:
                pass
lons = nc.sel(sv = 'C03').lon.dropna(dim = 'observer').values
lons = sorted(lons, reverse = True)

observers_sorted = []
for lon in lons:
    for observer in observers:
        if nc.lon.sel(observer = observer, sv = 'C03').values == lon:
            observers_sorted.append(observer)
            print(observer)

del nc

# observer_svs['HBES'].remove('C04')
# observers.remove('SCLT')
# observers.remove('SCJU')
# observers.remove('SCMN')
# observers = {'SCYX'}

ticks = [13, 1]

levels_low = np.arange(1, 25, 1.5)    # low frequencies
levels_medium = np.arange(0.2, 2.7, 0.1)  # medium frequencies
levels_high = np.arange(0.3, 2.2, 0.1)  # high frequencies
levels_higher = np.arange(0.1, 1.5, 0.1)  # higher frequencies
levels_muchhigher = np.arange(0.01, 0.25, 0.02)  # muchhigher frequencies
    
chose = 'higher'
for obs_count, observer in enumerate(observers_sorted):
    
    
    
    TEC_savefile  = save_dir/f'detrended_vTEC_{observer}.png'
    low_savefile  = save_dir/f'CWT_lowfrequencies_detrended_vTEC_{observer}.png'
    medium_savefile  = save_dir/f'CWT_higherfrequencies_detrended_vTEC_{observer}.png'
    higher_savefile  = save_dir/f'CWT_muchhigherfrequencies_detrended_vTEC_{observer}.png'
    
#     save_file        = save_dir/f'CWT_muchhigherfrequencies_detrended_vTEC_{observer}.png'
    
    if chose == 'TEC':
        save_file = TEC_savefile
            
    if chose == 'low':
        save_file = low_savefile
        
    if chose == 'medium':
        save_file = medium_savefile
        
    if chose == 'higher':
        save_file = higher_savefile
        
#     if Path.is_file(save_file):
#         continue
# |         |         |
# | ------- | ------- |
# |         |         |
# | medium_ | higher_ |
# |         |         |


    print(f'| {observer} |  low  |\n'
          f'| ------- | ------- |\n'
          f'| ![detrended_vTEC_{observer}]({TEC_savefile}) | ![CWT_lowfrequencies_detrended_vTEC_{observer}]({low_savefile})|\n'
          f'| medium_ | higher_ |\n'
          f'| ![CWT_higherfrequencies_detrended_vTEC_{observer}]({medium_savefile}) | ![CWT_muchhigherfrequencies_detrended_vTEC_{observer}]({higher_savefile})|\n'
          f'\n'
         )
    continue
    
    print(len(observers_sorted), obs_count, observer)
#     count = 0
    
    sv_count = len(observer_svs[observer])
    for count, sv in enumerate(observer_svs[observer]):
        is_first_day = True
        for date in pd.date_range(start+timedelta(days=1), 
                                  end-timedelta(days=1), 
                                  freq="D"
                                 ):
            obs['year']      = date.strftime('%y')
            obs['doy']       = date.strftime('%j')
            obs['samp_rate'] = '01s'

            # src_dir  = Path(f"/run/media/echoo/TOSHIBA EXT")

            TEC_dir  = src_dir/obs['doy']/'TEC'
            TEC_file = f"TEC_{obs['doy']}_{obs['samp_rate']}_{obs['year']}.nc"
            TEC_path = TEC_dir/TEC_file
            nc = xr.open_dataset(TEC_path)
            nc.coords['time'] = nc.time + np.timedelta64(timezone, 'h')
            nc.attrs['time_sysm'] = 'BJT'
                
            sTEC   = nc.sTEC_smth.sel(observer = observer, sv = sv).dropna(dim = 'time') 
#             print(sTEC)
            zenith = nc.zenith.sel(observer = observer, sv = sv)
            vTECdaily    = sTEC * np.cos(zenith)

            if is_first_day:
                vTEC     = vTECdaily
            else:
                vTEC      = xr.concat([vTEC  , vTECdaily]    , dim = 'time')
            is_first_day = False
            
            del nc 
            gc.collect()
            
            if len(vTEC) == 0:
                break
                                   
        data         = vTEC
        time         = vTEC.time
        t            = np.arange(1,len(time)+1,1)
        std          = data.std()

        p            = np.polyfit(t, data, 2)
        trend        = np.polyval(p, t)
        dtrend       = data - trend
        data_norm    = dtrend / std

        [cwtmatr, frequencies] = pywt.cwt(data_norm, 
                                          scales, 
                                          wavelet, 
                                          1.0, 
                                          'fft'
                                         )

        periods = np.power(frequencies, -1)

        xstart = AEindex['time'].values[0]
        xend   = AEindex['time'].values[-1] + np.timedelta64(1, 'm')
        
        if count == 0:
            fig = figure(figsize = (8, 12))
            axs = fig.subplots(3+sv_count, 1 )
#             fig, axs = plt.subplots(3+sv_count, 1, figsize = (8, 12))
        
        mark = chr(ord('d')+count)
        elevation  = 90 - np.degrees(zenith.values)
        
        if chose == 'TEC':
            plotTECnorm(axs[3+count], time, dtrend, mark)
            
        if chose == 'low':
            levels = levels_low
            plotCWT(axs[3+count], time, periods, frequencies, cwtmatr, mark, levels)
        
        if chose == 'medium':
            levels = levels_higher
            plotCWT(axs[3+count], time, periods, frequencies, cwtmatr, mark, levels)
            
        if chose == 'higher':
            levels = levels_muchhigher
            plotCWT(axs[3+count], time, periods, frequencies, cwtmatr, mark, levels)
        
        
#         plotTECnorm(axs[3+count], time, dtrend, mark)
        
#         plotCWT(axs[3+count], time, periods, frequencies, cwtmatr, mark)
        
        del cwtmatr, periods, frequencies
        gc.collect()
        
        axs[3+count].set_title(f'{dtrend.observer.values} '
                               f'{dtrend.sv.values}: '
                               f'{dtrend.lon.values:6.2f}\N{DEGREE SIGN} '
                               f'{dtrend.lat.values:5.2f}\N{DEGREE SIGN} '
                               f'{elevation:5.2f}\N{DEGREE SIGN}', 
                               loc = 'right'
                              )
        
#         axs[3+count]
        
    plotF107(axs[0], 
             indices
            )

    plotAE(axs[1], 
           AEindex
          )

    plotKp(axs[2], 
           indices
          )

    for ax in axs:
        ax.set_xlim(xstart, xend)
    

#     ticks = [13, 1]
    axs[-1].xaxis.set_major_locator(mdates.HourLocator(ticks))
    axs[-1].xaxis.set_minor_locator(mdates.HourLocator())
    axs[-1].xaxis.set_major_formatter(format_func)
    axs[-1].set_xlabel('time (LT)')

                                   
    fig.subplots_adjust(hspace = 0.35)
    fig.align_ylabels()

    fig.savefig(save_file,
                bbox_inches = 'tight'
               )
    
    matplotlib.use('Agg')
#     plt.show(fig)
    
    fig.clf()
#     fig.cla()
    plt.clf()
    plt.close('all')
    plt.close(fig)
    
#     pylab.close(fig)
#     plt.clf()
#     plt.close('all')
    
    
    del fig, axs, vTEC, data
    gc.collect()


SCJU
SCMB
SCTQ
SCYX
SCXD
SCXJ
SCMN
SCYY
SCDA
SCLT
SCXC
XZCY
XZBG
| SCJU |  low  |
| ------- | ------- |
| ![detrended_vTEC_SCJU](/home/echoo/echoo/mycode/PLOT/2022_01_15_Tonga/detrended_vTEC_SCJU.png) | ![CWT_lowfrequencies_detrended_vTEC_SCJU](/home/echoo/echoo/mycode/PLOT/2022_01_15_Tonga/CWT_lowfrequencies_detrended_vTEC_SCJU.png)|
| medium_ | higher_ |
| ![CWT_higherfrequencies_detrended_vTEC_SCJU](/home/echoo/echoo/mycode/PLOT/2022_01_15_Tonga/CWT_higherfrequencies_detrended_vTEC_SCJU.png) | ![CWT_muchhigherfrequencies_detrended_vTEC_SCJU](/home/echoo/echoo/mycode/PLOT/2022_01_15_Tonga/CWT_muchhigherfrequencies_detrended_vTEC_SCJU.png)|


| SCMB |  low  |
| ------- | ------- |
| ![detrended_vTEC_SCMB](/home/echoo/echoo/mycode/PLOT/2022_01_15_Tonga/detrended_vTEC_SCMB.png) | ![CWT_lowfrequencies_detrended_vTEC_SCMB](/home/echoo/echoo/mycode/PLOT/2022_01_15_Tonga/CWT_lowfrequencies_detrended_vTEC_SCMB.png)|
| medium_ | higher_ |
| ![CWT_higherfrequencies_detrended_vTEC_SCMB](/home/e

In [ ]:
# import pandas as pd
# from sys import getsizeof
# def get_memory(threshold=1048576):
#     '''查看变量占用内存情况

#     :param threshold: 仅显示内存数值大于等于threshold的变量, 默认为 1MB=1048576B
#     '''
#     memory_df=pd.DataFrame(columns=['name', 'memory', 'convert_memory'])
#     i=0
#     for key in list(globals().keys()):
#         memory = eval("getsizeof({})".format(key))
#         if memory<threshold:
#             continue
#         if(memory>1073741824):# GB
#             unit='GB'
#             convert_memory=round(memory/1073741824)
#         elif(memory>1048576):# MB
#             unit='MB'
#             convert_memory=round(memory/1048576)           
#         elif(memory>1024):# KB
#             unit='KB'
#             convert_memory=round(memory/1024)  
#         else:
#             unit='B' 
#             convert_memory = memory
#         memory_df.loc[i]=[key, memory, str(convert_memory)+unit]
#         i=i+1
#     # 按照内存占用大小降序排序    
#     memory_df.sort_values("memory",inplace=True,ascending=False)
#     return memory_df
    
# memory_df = get_memory()
# memory_df

In [ ]:
sorted(lons, reverse = True)

In [15]:
nc

<xarray.Dataset>
Dimensions:    (sv: 4, time: 102791, observer: 13)
Coordinates:
  * sv         (sv) object 'C01' 'C02' 'C03' 'C05'
  * time       (time) datetime64[ns] 2022-01-15T19:26:13 ... 2022-01-16T23:59:59
  * observer   (observer) object 'XZCY' 'XZBG' 'SCYY' ... 'SCLT' 'SCJU' 'SCDA'
    lon        (observer, sv) float64 ...
    lat        (observer, sv) float64 ...
Data variables:
    sTEC_smth  (observer, sv, time) float64 ...
    zenith     (observer, sv) float64 ...
    DCBs       (observer) float64 ...
Attributes:
    sample_rate:  1.0
    Geo_sysm:     WGS 84
    time_sysm:    BDT
    rxmodel:      TRIMBLE NETR9
    combination:  E2_E5b_I